# 構造解析 GNN ツールキット

VTU（FEA解析結果）から GNN を自動構築・学習し、任意荷重での応力・変位を推論するツール。  
**GUI モード**（タブ操作）と **コードモード**（Python API）の両方で利用可能。

> セル1（セットアップ）→ セル2（GUI）or セル3（コード）の順で実行してください。

## セル1: 環境セットアップ & インポート

依存パッケージの確認と `gnn_toolkit` パッケージの読み込みを行います。  
- **ローカル**: `gnn_toolkit/` と `data/` フォルダが同じディレクトリにあれば OK  
- **Colab**: リポジトリを `git clone` して `gnn_toolkit/` と学習データ（`data/`）を自動展開します  

| フォルダ | 用途 |
|---------|------|
| `data/` | 学習用 VTU ファイル |
| `results/` | 推論結果の VTU ファイル |
| `saved_model/` | 学習済みモデル |

In [4]:
# === 環境セットアップ & インポート ===
import sys, subprocess, os

# --- 依存パッケージの確認 ---
def _install_if_missing(pkg, pip_name=None):
    try:
        __import__(pkg)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pip_name or pkg])

_install_if_missing("torch")
_install_if_missing("torch_geometric", "torch_geometric")
_install_if_missing("pyvista")
_install_if_missing("ipywidgets")

# --- Colab 対応: リポジトリから gnn_toolkit/ と data/ を展開 ---
try:
    from google.colab import output
    output.enable_custom_widget_manager()
    if not os.path.isdir("gnn_toolkit"):
        subprocess.check_call(["git", "clone", "--depth", "1",
                               "https://github.com/WOCae/R231.git", "_repo_tmp"])
        import shutil
        shutil.copytree("_repo_tmp/gnn_toolkit", "gnn_toolkit")
        shutil.copytree("_repo_tmp/data", "data")
        if os.path.isdir("_repo_tmp/saved_model"):
            shutil.copytree("_repo_tmp/saved_model", "saved_model")
        shutil.rmtree("_repo_tmp")
    os.makedirs("results", exist_ok=True)
    print("[Colab] gnn_toolkit/ + data/ 展開完了 ✓")
    print(f"  data/: {os.listdir('data')}")
except ImportError:
    pass  # ローカル環境

# --- フォルダ作成（ローカル用） ---
os.makedirs("data", exist_ok=True)
os.makedirs("results", exist_ok=True)

# --- ツールキット読み込み ---
from gnn_toolkit import GNNToolkit, GNNToolkitUI

print("セットアップ完了 ✓")

セットアップ完了 ✓


## セル2: GUI モード
タブ付きの操作パネルで **学習 → 推論 → 評価 → 保存/読込** を行えます。

In [5]:
ui = GNNToolkitUI()
ui.show()

## セル3: コードモード（Python API）
GUI を使わず、Python コードで直接操作する場合はこちらを使用します。

In [6]:
# --- 学習（data/ フォルダ内の VTU を参照） ---
tk = GNNToolkit(train_load=1000.0)
tk.train("1_1000.vtu")          # data/1_1000.vtu を自動検出

# --- 精度評価 ---
tk.evaluate("1_1000.vtu")

# --- 推論（結果は results/ に保存） ---
tk.predict("1_1000.vtu", load_N=500.0)    # → results/gnn_500N_result.vtu
tk.predict("1_1000.vtu", load_N=1800.0)   # → results/gnn_1800N_result.vtu

# --- モデル保存 ---
tk.save("saved_model")

[GNNToolkit] device = cpu
  data_dir    = c:\work\HP4\GNN\R231\data
  results_dir = c:\work\HP4\GNN\R231\results
[Step 1] データ解析 & 自動キャリブレーション
=== data\1_1000.vtu ===
  ノード: 282
  要素: 103
  範囲: BoundsTuple(x_min =   0.0,
            x_max =  10.0,
            y_min =   0.0,
            y_max =  10.0,
            z_min =   0.0,
            z_max = 100.0)
  Point: ['Displacement', 'Displacement Magnitude', 'Tresca Stress', 'Strain xx component', 'Strain xy component', 'Strain xz component', 'Strain yy component', 'Strain yz component', 'Strain zz component', 'Stress xx component', 'Stress xy component', 'Stress xz component', 'Stress yy component', 'Stress yz component', 'Stress zz component', 'Major Principal Stress', 'Intermediate Principal Stress', 'Minor Principal Stress', 'von Mises Stress']
  Cell: []
  座標正規化  : 100.0
  変位キー    : Displacement  (正規化 = 0.00476156 mm)
  応力キー    : von Mises Stress  (正規化 = 10.00019999964556 MPa)
  基準荷重    : 1000.0 N
  モデル      : 4層 × hidden=128
  学習     

## セル4: 保存済みモデルの読込・再推論
学習済みモデルを読み込んで、すぐに推論を実行できます。

In [7]:
tk = GNNToolkit()
tk.load("saved_model")
tk.predict("1_1000.vtu", load_N=1200.0)

[GNNToolkit] device = cpu
  data_dir    = c:\work\HP4\GNN\R231\data
  results_dir = c:\work\HP4\GNN\R231\results
[読込完了] saved_model/

--- [1200.0N 推論結果] ---
  最大変位 : 0.00574 mm
  最大応力 : 12.03118 MPa
  保存先   : results\gnn_1200N_result.vtu


{'max_disp': 0.0057390243746340275,
 'max_stress': 12.031177520751953,
 'output': 'results\\gnn_1200N_result.vtu'}